CODE FROM https://becominghuman.ai/siamese-networks-algorithm-applications-and-pytorch-implementation-4ffa3304c18 
Classifying MNIST Images Using A Siamese Network In PyTorch

The aim of the network here is differentiating images, i.e. outputting "True" if a given pair of images represent the same number and "False" otherwise. 

In [87]:
#########################################
#          ALL THE IMPORTS              #
#########################################

import codecs
import errno
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import random
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torchvision.datasets.mnist
from torchvision.datasets.mnist import read_image_file, read_label_file
from torchvision import transforms
from tqdm import tqdm

In [88]:
#########################################
#       ALL THE GENRAL CONSTANTS        #
#########################################
do_learn = True
save_frequency = 2
batch_size = 4 # => At each iteration: 160 inputs (among 5000) are considered 
lr = 0.001
num_epochs = 10 
weight_decay = 0.0001

In [89]:
#########################################
#       CLASS BALANCED MNISTPAIR        #
#########################################

class BalancedMNISTPair(torch.utils.data.Dataset):
   """ ---------------------------------------------------------------
   Dataset that on each iteration provides two random pairs of
   MNIST images. One pair is of the same number (positive sample), one
   is of two different numbers (negative sample).
   BalancedMNISTPair herits of all the methods and variables of 
   torch.utils.data.Dataset
   ---------------------------------------------------------------"""
   urls = [
      'http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
      'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
      'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
      'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz',
   ]
   raw_folder = 'raw'
   processed_folder = 'processed'
   training_file = 'training.pt'
   test_file = 'test.pt'
   
   def __init__(self, root, train=True, transform=None, target_transform=None, download=False):
      self.root = os.path.expanduser(root)
      self.transform = transform
      self.target_transform = target_transform
      self.train = train # training set or test set
      
      if download:
         self.download()
         
      if not self._check_exists():
         raise RuntimeError('Dataset not found.' + ' You can use download=True to download it')
         
        
         ###########################################
         # FOR TRAINING
         ###########################################

      if self.train:
         self.train_data, self.train_labels = torch.load(
            os.path.join(self.root, self.processed_folder, self.training_file))
         print("The type of train_data 1 is " + str(type(self.train_data)))
         print("The type of train_labels 1 is " + str(type(self.train_labels)))
         print("The train_labels 1 is " + str(self.train_labels))

         train_labels_class = []
         train_data_class = []
         for i in range(10):
            indices = torch.squeeze((self.train_labels == i).nonzero())
            train_labels_class.append(torch.index_select(self.train_labels, 0, indices))
            train_data_class.append(torch.index_select(self.train_data, 0, indices))
            
         ###########################################
         # Generate balanced pairs
         ###########################################

         self.train_data = []
         self.train_labels = []
         lengths = [x.shape[0] for x in train_labels_class]
         for i in range(10):
            for j in range(500): # create 500 pairs
               rnd_cls = random.randint(0,8) # choose random class that is not the same class
               if rnd_cls >= i:
                  rnd_cls = rnd_cls + 1

               rnd_dist = random.randint(0, 100)
                  
               self.train_data.append(torch.stack([train_data_class[i][j], train_data_class[i][j+rnd_dist], train_data_class[rnd_cls][j]]))
               self.train_labels.append([1,0])

         print("The type of train_data 2 is " + str(type(self.train_data)))
         print("The type of train_labels 2 is " + str(type(self.train_labels)))
            
         self.train_data = torch.stack(self.train_data)
         self.train_labels = torch.tensor(self.train_labels)

         print("The type of train_data 3 is " + str(type(self.train_data)))
         print("The type of train_labels 3 is " + str(type(self.train_labels)))
           
        
         ###########################################
         # FOR TESTING 
         ###########################################

      else:
         self.test_data, self.test_labels = torch.load(
            os.path.join(self.root, self.processed_folder, self.test_file))
         
         test_labels_class = []
         test_data_class = []
         for i in range(10):
            indices = torch.squeeze((self.test_labels == i).nonzero())
            test_labels_class.append(torch.index_select(self.test_labels, 0, indices))
            test_data_class.append(torch.index_select(self.test_data, 0, indices))
            
            
         ###########################################
         # Generate balanced pairs
         ###########################################
         self.test_data = []
         self.test_labels = []
         lengths = [x.shape[0] for x in test_labels_class]
         for i in range(10):
            for j in range(500): # create 500 pairs
               rnd_cls = random.randint(0,8) # choose random class that is not the same class
               if rnd_cls >= i:
                  rnd_cls = rnd_cls + 1

               rnd_dist = random.randint(0, 100)
                  
               self.test_data.append(torch.stack([test_data_class[i][j], test_data_class[i][j+rnd_dist], test_data_class[rnd_cls][j]]))
               self.test_labels.append([1,0])

         self.test_data = torch.stack(self.test_data)
         self.test_labels = torch.tensor(self.test_labels)
         print("1 test data is " + str(self.test_data[1]))
         print("1 test label is " + str(self.test_labels[1]))

         
   def __getitem__(self, index):
      if self.train:
            imgs, target = self.train_data[index], self.train_labels[index]
      else:
        imgs, target = self.test_data[index], self.test_labels[index]
      img_ar = []
      for i in range(len(imgs)):
         img = Image.fromarray(imgs[i].numpy(), mode='L')
         if self.transform is not None:
            img = self.transform(img)
            print("IN GETITEM: the type of image is " + str(type(img)))
         img_ar.append(img)
         
      if self.target_transform is not None:
         target = self.target_transform(target)
         
      return img_ar, target
   
   def __len__(self):
      if self.train:
         return len(self.train_data)
      else:
         return len(self.test_data)
      
   def _check_exists(self):
      return os.path.exists(os.path.join(self.root, self.processed_folder, self.training_file)) and \
         os.path.exists(os.path.join(self.root, self.processed_folder, self.test_file))
   
   def download(self):
      """Download the MNIST data if it doesn't exist in processed_folder already."""
      from six.moves import urllib
      import gzip

      if self._check_exists():
         return

      # download files
      try:
         os.makedirs(os.path.join(self.root, self.raw_folder))
         os.makedirs(os.path.join(self.root, self.processed_folder))
      except OSError as e:
         if e.errno == errno.EEXIST:
            pass
         else:
            raise

      for url in self.urls:
         print('Downloading ' + url)
         data = urllib.request.urlopen(url)
         filename = url.rpartition('/')[2]
         file_path = os.path.join(self.root, self.raw_folder, filename)
         with open(file_path, 'wb') as f:
            f.write(data.read())
         with open(file_path.replace('.gz', ''), 'wb') as out_f, \
               gzip.GzipFile(file_path) as zip_f:
            out_f.write(zip_f.read())
         os.unlink(file_path)

      # process and save as torch files
      print('Processing in download function...')

      training_set = (
         read_image_file(os.path.join(self.root, self.raw_folder, 'train-images-idx3-ubyte')),
         read_label_file(os.path.join(self.root, self.raw_folder, 'train-labels-idx1-ubyte'))
      )
      test_set = (
         read_image_file(os.path.join(self.root, self.raw_folder, 't10k-images-idx3-ubyte')),
         read_label_file(os.path.join(self.root, self.raw_folder, 't10k-labels-idx1-ubyte'))
      )
      with open(os.path.join(self.root, self.processed_folder, self.training_file), 'wb') as f:
         torch.save(training_set, f)
      with open(os.path.join(self.root, self.processed_folder, self.test_file), 'wb') as f:
         torch.save(test_set, f)

      print('Download of the data is Done!')

   def __repr__(self):
      fmt_str = 'Dataset ' + self.__class__.__name__ + '\n'
      fmt_str += '    Number of datapoints: {}\n'.format(self.__len__())
      tmp = 'train' if self.train is True else 'test'
      fmt_str += '    Split: {}\n'.format(tmp)
      fmt_str += '    Root Location: {}\n'.format(self.root)
      tmp = '    Transforms (if any): '
      fmt_str += '{0}{1}\n'.format(tmp, self.transform.__repr__().replace('\n', '\n' + ' ' * len(tmp)))
      tmp = '    Target Transforms (if any): '
      fmt_str += '{0}{1}'.format(tmp, self.target_transform.__repr__().replace('\n', '\n' + ' ' * len(tmp)))
      return fmt_str

In [90]:
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])

ds = BalancedMNISTPair('../data', train=False, download=True, transform=trans)
ds.__getitem__(9)


1 test data is tensor([[[  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        [[  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        [[  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]]], dtype=torch.uint8)
1 test label is tensor([1, 0])
IN GETITEM: the type of image is <class '

([tensor([[[-0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
            -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
            -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
            -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000],
           [-0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
            -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
            -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
            -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000],
           [-0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
            -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
            -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
            -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000],
           [-0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
            -0.5000, -

In [91]:
#########################################
#       CLASS Net                       #
#########################################
class Net(nn.Module):
   def __init__(self):
      super().__init__()
      
      self.conv1 = nn.Conv2d(1, 64, 7)
      self.pool1 = nn.MaxPool2d(2)
      self.conv2 = nn.Conv2d(64, 128, 5)
      self.conv3 = nn.Conv2d(128, 256, 5)
      self.linear1 = nn.Linear(2304, 512)
      
      self.linear2 = nn.Linear(512, 2) # 512 output features 
      
   def forward(self, data):
      res = []
      for i in range(2): # Siamese nets; sharing weights
         x = data[i]
         x = self.conv1(x)
         x = F.relu(x)
         x = self.pool1(x)
         x = self.conv2(x)
         x = F.relu(x)
         x = self.conv3(x)
         x = F.relu(x)
         
         x = x.view(x.shape[0], -1)
         x = self.linear1(x)
         res.append(F.relu(x))
         
      res = torch.abs(res[1] - res[0])
      res = self.linear2(res)
      return res

In [92]:
#########################################
#       FUNCTION train                  #
#########################################

def train(model, device, train_loader, epoch, optimizer):
   model.train()
   
   for batch_idx, (data, target) in enumerate(train_loader):
      print("IN TRAIN: the data is " + str(data))
      for i in range(len(data)):
         print("IN TRAIN: 1 data is " + str(data[i]))
         data[i] = data[i].to(device)
         print("IN TRAIN: 1 data after application to device is " + str(data[i]))

         
      optimizer.zero_grad()
      output_positive = model(data[:2]) # Take the 2 first images corresponding to the positive couple
      output_negative = model(data[0:3:2]) # Take the first and the third images corresponding to the negative couple
      
      target = target.type(torch.LongTensor).to(device)
      print("IN TRAIN: the type of target is " + str(type(target)))
      print("IN TRAIN: the target is " + str(target))

      target_positive = torch.squeeze(target[:,0]) # Removes 1 dimension 
      target_negative = torch.squeeze(target[:,1])
      
      loss_positive = F.cross_entropy(output_positive, target_positive)
      loss_negative = F.cross_entropy(output_negative, target_negative)
      
      loss = loss_positive + loss_negative
      loss.backward() # Minimization 
      
      optimizer.step()
      if batch_idx % 10 == 0:
         print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx*batch_size, len(train_loader.dataset), 100. * batch_idx*batch_size / len(train_loader.dataset),
            loss.item()))



In [93]:
#########################################
#       FUNCTION test                   #
#########################################

def test(model, device, test_loader):
   model.eval()
   
   with torch.no_grad():
      accurate_labels = 0
      all_labels = 0
      loss = 0
      for batch_idx, (data, target) in enumerate(test_loader):
         for i in range(len(data)):
            data[i] = data[i].to(device)
            
         output_positive = model(data[:2])
         output_negative = model(data[0:3:2])
            
         target = target.type(torch.LongTensor).to(device)
         target_positive = torch.squeeze(target[:,0])
         target_negative = torch.squeeze(target[:,1])
            
         loss_positive = F.cross_entropy(output_positive, target_positive)
         loss_negative = F.cross_entropy(output_negative, target_negative)
            
         loss = loss + loss_positive + loss_negative
            
         accurate_labels_positive = torch.sum(torch.argmax(output_positive, dim=1) == target_positive).cpu()
         accurate_labels_negative = torch.sum(torch.argmax(output_negative, dim=1) == target_negative).cpu()
            
         accurate_labels = accurate_labels + accurate_labels_positive + accurate_labels_negative
         all_labels = all_labels + len(target_positive) + len(target_negative)
      
      accuracy = 100. * accurate_labels / all_labels
      print('Test accuracy: {}/{} ({:.3f}%)\tLoss: {:.6f}'.format(accurate_labels, all_labels, accuracy, loss))
   

In [94]:
#########################################
#       FUNCTION oneshot                #
#########################################

def oneshot(model, device, data):
   model.eval()

   with torch.no_grad():
      for i in range(len(data)):
            data[i] = data[i].to(device)
      
      output = model(data)
      return torch.squeeze(torch.argmax(output, dim=1)).cpu().item()


In [95]:
#########################################
#       FUNCTION main                   #
#########################################

def main():
   device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
   trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
   name_model = "siamese_"
   extension_model = ".pt"
   
   model = Net().to(device)
   
   if do_learn: # training mode
      train_loader = torch.utils.data.DataLoader(BalancedMNISTPair('../data', train=True, download=True, transform=trans), batch_size=batch_size, shuffle=True)
      test_loader = torch.utils.data.DataLoader(BalancedMNISTPair('../data', train=False, download=True, transform=trans), batch_size=batch_size, shuffle=False)
      
      optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
      for epoch in range(num_epochs):
         train(model, device, train_loader, epoch, optimizer)
         test(model, device, test_loader)
         if epoch & save_frequency == 0:
            torch.save(model, (name_model + '{:03}' + extension_model).format(epoch))
            print("Model is saved!")
      
   else: # prediction
      dataset = BalancedMNISTPair('../data', train=False, download=True, transform=trans)
      #print(dataset.__repr__)
      prediction_loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)
      
      load_model_path = os.getcwd() + "/" + name_model + "000" + extension_model
      #model.load_state_dict(torch.load(load_model_path))
      model = torch.load(load_model_path)

      #####################################################################
      # Data: list containing the tensor representations of the 2 images
      #####################################################################
      data = []
      data.extend(next(iter(prediction_loader))[0][:3:2])
      same = oneshot(model, device, data)
      if same > 0:
         print('These two images are of the same number')
      else:
         print('These two images are not of the same number')
    

In [96]:
if __name__ == '__main__':
   main()

The type of train_data 1 is <class 'torch.Tensor'>
The type of train_labels 1 is <class 'torch.Tensor'>
The train_labels 1 is tensor([5, 0, 4,  ..., 5, 6, 8])
The type of train_data 2 is <class 'list'>
The type of train_labels 2 is <class 'list'>
The type of train_data 3 is <class 'torch.Tensor'>
The type of train_labels 3 is <class 'torch.Tensor'>
1 test data is tensor([[[  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        [[  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        [[  0,   0,   0,  ...,   0,   

IN TRAIN: 1 data after application to device is tensor([[[[-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          ...,
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000]]],


        [[[-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          ...,
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000]]],


        [[[-0.5000, -0.5000, -0.5000,  ..., -0.500

Train Epoch: 0 [0/5000 (0%)]	Loss: 1.387525
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN TRAIN: the data is [tensor([[[[-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          ...,
          [-0.5000, -0.5000, -

IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN TRAIN: the data is [tensor([[[[-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          ...,
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
  

IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN TRAIN: the data is [tensor([[[[-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          ...,
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
  

...

IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN TRAIN: the data is [tensor([[[[-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          ...,
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
  

IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN TRAIN: the data is [tensor([[[[-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          ...,
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ...

Train Epoch: 0 [200/5000 (4%)]	Loss: 1.550276
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN TRAIN: the data is [tensor([[[[-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          ...,
          [-0.5000, -0.5000,

IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN GETITEM: the type of image is <class 'torch.Tensor'>
IN TRAIN: the data is [tensor([[[[-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
          ...,
          [-0.5000, -0.5000, -0.5000,  ..., -0.5000, -0.5000, -0.5000],
  

KeyboardInterrupt: 

OUTPUT OF RUN 1 (nb_epoch = 10)

Downloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
Downloading http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
Downloading http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
Downloading http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
Processing...
Done!
Train Epoch: 0 [0/5000 (0%)]	Loss: 1.386692
Train Epoch: 0 [160/5000 (3%)]	Loss: 1.202920
Train Epoch: 0 [320/5000 (6%)]	Loss: 1.362132
Train Epoch: 0 [480/5000 (10%)]	Loss: 1.235610
Train Epoch: 0 [640/5000 (13%)]	Loss: 1.192515
Train Epoch: 0 [800/5000 (16%)]	Loss: 1.380266
Train Epoch: 0 [960/5000 (19%)]	Loss: 1.106000
Train Epoch: 0 [1120/5000 (22%)]	Loss: 0.919552
Train Epoch: 0 [1280/5000 (26%)]	Loss: 0.950035
Train Epoch: 0 [1440/5000 (29%)]	Loss: 1.070726
Train Epoch: 0 [1600/5000 (32%)]	Loss: 0.688442
Train Epoch: 0 [1760/5000 (35%)]	Loss: 1.538752
Train Epoch: 0 [1920/5000 (38%)]	Loss: 0.654107
Train Epoch: 0 [2080/5000 (42%)]	Loss: 0.832105
Train Epoch: 0 [2240/5000 (45%)]	Loss: 0.802137
Train Epoch: 0 [2400/5000 (48%)]	Loss: 0.903138
Train Epoch: 0 [2560/5000 (51%)]	Loss: 1.166157
Train Epoch: 0 [2720/5000 (54%)]	Loss: 0.798701
Train Epoch: 0 [2880/5000 (58%)]	Loss: 0.753121
Train Epoch: 0 [3040/5000 (61%)]	Loss: 0.537702
Train Epoch: 0 [3200/5000 (64%)]	Loss: 0.756473
Train Epoch: 0 [3360/5000 (67%)]	Loss: 0.754277
Train Epoch: 0 [3520/5000 (70%)]	Loss: 0.566075
Train Epoch: 0 [3680/5000 (74%)]	Loss: 0.518971
Train Epoch: 0 [3840/5000 (77%)]	Loss: 0.763466
Train Epoch: 0 [4000/5000 (80%)]	Loss: 0.938381
Train Epoch: 0 [4160/5000 (83%)]	Loss: 0.589798
Train Epoch: 0 [4320/5000 (86%)]	Loss: 0.642045
Train Epoch: 0 [4480/5000 (90%)]	Loss: 0.731936
Train Epoch: 0 [4640/5000 (93%)]	Loss: 0.371344
Train Epoch: 0 [4800/5000 (96%)]	Loss: 0.706093
Train Epoch: 0 [4960/5000 (99%)]	Loss: 0.425127
Test accuracy: 8187/10000 (81.000%)	Loss: 270.004089
/Users/geraldinebrieven/anaconda3/lib/python3.7/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
Train Epoch: 1 [0/5000 (0%)]	Loss: 1.157182
Train Epoch: 1 [160/5000 (3%)]	Loss: 0.657840
Train Epoch: 1 [320/5000 (6%)]	Loss: 0.524534
Train Epoch: 1 [480/5000 (10%)]	Loss: 0.225388
Train Epoch: 1 [640/5000 (13%)]	Loss: 0.772241
Train Epoch: 1 [800/5000 (16%)]	Loss: 0.474888
Train Epoch: 1 [960/5000 (19%)]	Loss: 0.610854
Train Epoch: 1 [1120/5000 (22%)]	Loss: 0.412593
Train Epoch: 1 [1280/5000 (26%)]	Loss: 0.267956
Train Epoch: 1 [1440/5000 (29%)]	Loss: 0.616167
Train Epoch: 1 [1600/5000 (32%)]	Loss: 0.501102
Train Epoch: 1 [1760/5000 (35%)]	Loss: 0.431408
Train Epoch: 1 [1920/5000 (38%)]	Loss: 0.864756
Train Epoch: 1 [2080/5000 (42%)]	Loss: 0.619375
Train Epoch: 1 [2240/5000 (45%)]	Loss: 0.320641
Train Epoch: 1 [2400/5000 (48%)]	Loss: 0.277328
Train Epoch: 1 [2560/5000 (51%)]	Loss: 0.511122
Train Epoch: 1 [2720/5000 (54%)]	Loss: 0.200149
Train Epoch: 1 [2880/5000 (58%)]	Loss: 0.249798
Train Epoch: 1 [3040/5000 (61%)]	Loss: 0.449537
Train Epoch: 1 [3200/5000 (64%)]	Loss: 0.338033
Train Epoch: 1 [3360/5000 (67%)]	Loss: 0.513476
Train Epoch: 1 [3520/5000 (70%)]	Loss: 0.343755
Train Epoch: 1 [3680/5000 (74%)]	Loss: 0.300294
Train Epoch: 1 [3840/5000 (77%)]	Loss: 0.245068
Train Epoch: 1 [4000/5000 (80%)]	Loss: 0.826999
Train Epoch: 1 [4160/5000 (83%)]	Loss: 0.308307
Train Epoch: 1 [4320/5000 (86%)]	Loss: 0.099211
Train Epoch: 1 [4480/5000 (90%)]	Loss: 0.248848
Train Epoch: 1 [4640/5000 (93%)]	Loss: 0.261593
Train Epoch: 1 [4800/5000 (96%)]	Loss: 0.248067
Train Epoch: 1 [4960/5000 (99%)]	Loss: 0.420565
Test accuracy: 9164/10000 (91.000%)	Loss: 140.832184
Train Epoch: 2 [0/5000 (0%)]	Loss: 0.233463
Train Epoch: 2 [160/5000 (3%)]	Loss: 0.296097
Train Epoch: 2 [320/5000 (6%)]	Loss: 0.129859
Train Epoch: 2 [480/5000 (10%)]	Loss: 0.053269
Train Epoch: 2 [640/5000 (13%)]	Loss: 0.090427
Train Epoch: 2 [800/5000 (16%)]	Loss: 0.232051
Train Epoch: 2 [960/5000 (19%)]	Loss: 0.136354
Train Epoch: 2 [1120/5000 (22%)]	Loss: 0.104160
Train Epoch: 2 [1280/5000 (26%)]	Loss: 0.141883
Train Epoch: 2 [1440/5000 (29%)]	Loss: 0.816744
Train Epoch: 2 [1600/5000 (32%)]	Loss: 0.153644
Train Epoch: 2 [1760/5000 (35%)]	Loss: 0.417527
Train Epoch: 2 [1920/5000 (38%)]	Loss: 0.214715
Train Epoch: 2 [2080/5000 (42%)]	Loss: 0.358542
Train Epoch: 2 [2240/5000 (45%)]	Loss: 0.410211
Train Epoch: 2 [2400/5000 (48%)]	Loss: 0.077042
Train Epoch: 2 [2560/5000 (51%)]	Loss: 0.670398
Train Epoch: 2 [2720/5000 (54%)]	Loss: 0.203647
Train Epoch: 2 [2880/5000 (58%)]	Loss: 0.070777
Train Epoch: 2 [3040/5000 (61%)]	Loss: 0.408100
Train Epoch: 2 [3200/5000 (64%)]	Loss: 0.292756
Train Epoch: 2 [3360/5000 (67%)]	Loss: 0.175716
Train Epoch: 2 [3520/5000 (70%)]	Loss: 0.423782
Train Epoch: 2 [3680/5000 (74%)]	Loss: 0.243602
Train Epoch: 2 [3840/5000 (77%)]	Loss: 0.068839
Train Epoch: 2 [4000/5000 (80%)]	Loss: 0.122060
Train Epoch: 2 [4160/5000 (83%)]	Loss: 0.049606
Train Epoch: 2 [4320/5000 (86%)]	Loss: 0.276396
Train Epoch: 2 [4480/5000 (90%)]	Loss: 0.083387
Train Epoch: 2 [4640/5000 (93%)]	Loss: 0.268116
Train Epoch: 2 [4800/5000 (96%)]	Loss: 0.263652
Train Epoch: 2 [4960/5000 (99%)]	Loss: 0.107940
Test accuracy: 9448/10000 (94.000%)	Loss: 99.260063
Train Epoch: 3 [0/5000 (0%)]	Loss: 0.213459
Train Epoch: 3 [160/5000 (3%)]	Loss: 0.037763
Train Epoch: 3 [320/5000 (6%)]	Loss: 0.067553
Train Epoch: 3 [480/5000 (10%)]	Loss: 0.113720
Train Epoch: 3 [640/5000 (13%)]	Loss: 0.414817
Train Epoch: 3 [800/5000 (16%)]	Loss: 0.098427
Train Epoch: 3 [960/5000 (19%)]	Loss: 0.063124
Train Epoch: 3 [1120/5000 (22%)]	Loss: 0.050150
Train Epoch: 3 [1280/5000 (26%)]	Loss: 0.070426
Train Epoch: 3 [1440/5000 (29%)]	Loss: 0.023569
Train Epoch: 3 [1600/5000 (32%)]	Loss: 0.076250
Train Epoch: 3 [1760/5000 (35%)]	Loss: 0.028064
Train Epoch: 3 [1920/5000 (38%)]	Loss: 0.316921
Train Epoch: 3 [2080/5000 (42%)]	Loss: 0.215700
Train Epoch: 3 [2240/5000 (45%)]	Loss: 0.034618
Train Epoch: 3 [2400/5000 (48%)]	Loss: 0.044436
Train Epoch: 3 [2560/5000 (51%)]	Loss: 0.279604
Train Epoch: 3 [2720/5000 (54%)]	Loss: 0.135186
Train Epoch: 3 [2880/5000 (58%)]	Loss: 0.018894
Train Epoch: 3 [3040/5000 (61%)]	Loss: 0.504726
Train Epoch: 3 [3200/5000 (64%)]	Loss: 0.024579
Train Epoch: 3 [3360/5000 (67%)]	Loss: 0.164381
Train Epoch: 3 [3520/5000 (70%)]	Loss: 0.017969
Train Epoch: 3 [3680/5000 (74%)]	Loss: 0.371693
Train Epoch: 3 [3840/5000 (77%)]	Loss: 0.061414
Train Epoch: 3 [4000/5000 (80%)]	Loss: 0.395381
Train Epoch: 3 [4160/5000 (83%)]	Loss: 0.082733
Train Epoch: 3 [4320/5000 (86%)]	Loss: 0.053121
Train Epoch: 3 [4480/5000 (90%)]	Loss: 0.105092
Train Epoch: 3 [4640/5000 (93%)]	Loss: 0.191006
Train Epoch: 3 [4800/5000 (96%)]	Loss: 0.172529
Train Epoch: 3 [4960/5000 (99%)]	Loss: 0.035253
Test accuracy: 9488/10000 (94.000%)	Loss: 93.341164
Train Epoch: 4 [0/5000 (0%)]	Loss: 0.040333
Train Epoch: 4 [160/5000 (3%)]	Loss: 0.153183
Train Epoch: 4 [320/5000 (6%)]	Loss: 0.075854
Train Epoch: 4 [480/5000 (10%)]	Loss: 0.151485
Train Epoch: 4 [640/5000 (13%)]	Loss: 0.206435
Train Epoch: 4 [800/5000 (16%)]	Loss: 0.025918
Train Epoch: 4 [960/5000 (19%)]	Loss: 0.080630
Train Epoch: 4 [1120/5000 (22%)]	Loss: 0.058319
Train Epoch: 4 [1280/5000 (26%)]	Loss: 0.058575
Train Epoch: 4 [1440/5000 (29%)]	Loss: 0.301299
Train Epoch: 4 [1600/5000 (32%)]	Loss: 0.069123
Train Epoch: 4 [1760/5000 (35%)]	Loss: 0.079614
Train Epoch: 4 [1920/5000 (38%)]	Loss: 0.051245
Train Epoch: 4 [2080/5000 (42%)]	Loss: 0.025642
Train Epoch: 4 [2240/5000 (45%)]	Loss: 0.042222
Train Epoch: 4 [2400/5000 (48%)]	Loss: 0.117085
Train Epoch: 4 [2560/5000 (51%)]	Loss: 0.142101
Train Epoch: 4 [2720/5000 (54%)]	Loss: 0.130478
Train Epoch: 4 [2880/5000 (58%)]	Loss: 0.192402
Train Epoch: 4 [3040/5000 (61%)]	Loss: 0.036185
Train Epoch: 4 [3200/5000 (64%)]	Loss: 0.088661
Train Epoch: 4 [3360/5000 (67%)]	Loss: 0.008441
Train Epoch: 4 [3520/5000 (70%)]	Loss: 0.641124
Train Epoch: 4 [3680/5000 (74%)]	Loss: 0.085151
Train Epoch: 4 [3840/5000 (77%)]	Loss: 0.109239
Train Epoch: 4 [4000/5000 (80%)]	Loss: 0.025795
Train Epoch: 4 [4160/5000 (83%)]	Loss: 0.025071
Train Epoch: 4 [4320/5000 (86%)]	Loss: 0.043539
Train Epoch: 4 [4480/5000 (90%)]	Loss: 0.029609
Train Epoch: 4 [4640/5000 (93%)]	Loss: 0.017646
Train Epoch: 4 [4800/5000 (96%)]	Loss: 0.037034
Train Epoch: 4 [4960/5000 (99%)]	Loss: 0.025191
Test accuracy: 9404/10000 (94.000%)	Loss: 131.948120
Train Epoch: 5 [0/5000 (0%)]	Loss: 0.046703
Train Epoch: 5 [160/5000 (3%)]	Loss: 0.012659
Train Epoch: 5 [320/5000 (6%)]	Loss: 0.104272
Train Epoch: 5 [480/5000 (10%)]	Loss: 0.086894
Train Epoch: 5 [640/5000 (13%)]	Loss: 0.122875
Train Epoch: 5 [800/5000 (16%)]	Loss: 0.022615
Train Epoch: 5 [960/5000 (19%)]	Loss: 0.004790
Train Epoch: 5 [1120/5000 (22%)]	Loss: 0.080331
Train Epoch: 5 [1280/5000 (26%)]	Loss: 0.013124
Train Epoch: 5 [1440/5000 (29%)]	Loss: 0.254148
Train Epoch: 5 [1600/5000 (32%)]	Loss: 0.026347
Train Epoch: 5 [1760/5000 (35%)]	Loss: 0.287223
Train Epoch: 5 [1920/5000 (38%)]	Loss: 0.048779
Train Epoch: 5 [2080/5000 (42%)]	Loss: 0.029818
Train Epoch: 5 [2240/5000 (45%)]	Loss: 0.244076
Train Epoch: 5 [2400/5000 (48%)]	Loss: 0.077706
Train Epoch: 5 [2560/5000 (51%)]	Loss: 0.102413
Train Epoch: 5 [2720/5000 (54%)]	Loss: 0.005833
Train Epoch: 5 [2880/5000 (58%)]	Loss: 0.088083
Train Epoch: 5 [3040/5000 (61%)]	Loss: 0.104538
Train Epoch: 5 [3200/5000 (64%)]	Loss: 0.280958
Train Epoch: 5 [3360/5000 (67%)]	Loss: 0.041993
Train Epoch: 5 [3520/5000 (70%)]	Loss: 0.042134
Train Epoch: 5 [3680/5000 (74%)]	Loss: 0.029950
Train Epoch: 5 [3840/5000 (77%)]	Loss: 0.009111
Train Epoch: 5 [4000/5000 (80%)]	Loss: 0.013798
Train Epoch: 5 [4160/5000 (83%)]	Loss: 0.067221
Train Epoch: 5 [4320/5000 (86%)]	Loss: 0.024732
Train Epoch: 5 [4480/5000 (90%)]	Loss: 0.024272
Train Epoch: 5 [4640/5000 (93%)]	Loss: 0.057599
Train Epoch: 5 [4800/5000 (96%)]	Loss: 0.054665
Train Epoch: 5 [4960/5000 (99%)]	Loss: 0.004687
Test accuracy: 9561/10000 (95.000%)	Loss: 103.831314
Train Epoch: 6 [0/5000 (0%)]	Loss: 0.016730
Train Epoch: 6 [160/5000 (3%)]	Loss: 0.098461
Train Epoch: 6 [320/5000 (6%)]	Loss: 0.007959
Train Epoch: 6 [480/5000 (10%)]	Loss: 0.036200
Train Epoch: 6 [640/5000 (13%)]	Loss: 0.008279
Train Epoch: 6 [800/5000 (16%)]	Loss: 0.007751
Train Epoch: 6 [960/5000 (19%)]	Loss: 0.020641
Train Epoch: 6 [1120/5000 (22%)]	Loss: 0.003032
Train Epoch: 6 [1280/5000 (26%)]	Loss: 0.000461
Train Epoch: 6 [1440/5000 (29%)]	Loss: 0.003423
Train Epoch: 6 [1600/5000 (32%)]	Loss: 0.014104
Train Epoch: 6 [1760/5000 (35%)]	Loss: 0.031766
Train Epoch: 6 [1920/5000 (38%)]	Loss: 0.005861
Train Epoch: 6 [2080/5000 (42%)]	Loss: 0.122320
Train Epoch: 6 [2240/5000 (45%)]	Loss: 0.004101
Train Epoch: 6 [2400/5000 (48%)]	Loss: 0.203629
Train Epoch: 6 [2560/5000 (51%)]	Loss: 0.036162
Train Epoch: 6 [2720/5000 (54%)]	Loss: 0.059206
Train Epoch: 6 [2880/5000 (58%)]	Loss: 0.040846
Train Epoch: 6 [3040/5000 (61%)]	Loss: 0.005029
Train Epoch: 6 [3200/5000 (64%)]	Loss: 0.053877
Train Epoch: 6 [3360/5000 (67%)]	Loss: 0.307987
Train Epoch: 6 [3520/5000 (70%)]	Loss: 0.006260
Train Epoch: 6 [3680/5000 (74%)]	Loss: 0.029393
Train Epoch: 6 [3840/5000 (77%)]	Loss: 0.709541
Train Epoch: 6 [4000/5000 (80%)]	Loss: 0.257341
Train Epoch: 6 [4160/5000 (83%)]	Loss: 0.019037
Train Epoch: 6 [4320/5000 (86%)]	Loss: 0.122744
Train Epoch: 6 [4480/5000 (90%)]	Loss: 0.010510
Train Epoch: 6 [4640/5000 (93%)]	Loss: 0.021558
Train Epoch: 6 [4800/5000 (96%)]	Loss: 0.001194
Train Epoch: 6 [4960/5000 (99%)]	Loss: 0.146940
Test accuracy: 9488/10000 (94.000%)	Loss: 119.474754
Train Epoch: 7 [0/5000 (0%)]	Loss: 0.004329
Train Epoch: 7 [160/5000 (3%)]	Loss: 0.003300
Train Epoch: 7 [320/5000 (6%)]	Loss: 0.123778
Train Epoch: 7 [480/5000 (10%)]	Loss: 0.139458
Train Epoch: 7 [640/5000 (13%)]	Loss: 0.006652
Train Epoch: 7 [800/5000 (16%)]	Loss: 0.040077
Train Epoch: 7 [960/5000 (19%)]	Loss: 0.095122
Train Epoch: 7 [1120/5000 (22%)]	Loss: 0.017833
Train Epoch: 7 [1280/5000 (26%)]	Loss: 0.073878
Train Epoch: 7 [1440/5000 (29%)]	Loss: 0.030294
Train Epoch: 7 [1600/5000 (32%)]	Loss: 0.021073
Train Epoch: 7 [1760/5000 (35%)]	Loss: 0.018947
Train Epoch: 7 [1920/5000 (38%)]	Loss: 0.060272
Train Epoch: 7 [2080/5000 (42%)]	Loss: 0.103331
Train Epoch: 7 [2240/5000 (45%)]	Loss: 0.145447
Train Epoch: 7 [2400/5000 (48%)]	Loss: 0.100062
Train Epoch: 7 [2560/5000 (51%)]	Loss: 0.022028
Train Epoch: 7 [2720/5000 (54%)]	Loss: 0.005989
Train Epoch: 7 [2880/5000 (58%)]	Loss: 0.011031
Train Epoch: 7 [3040/5000 (61%)]	Loss: 0.097262
Train Epoch: 7 [3200/5000 (64%)]	Loss: 0.013701
Train Epoch: 7 [3360/5000 (67%)]	Loss: 0.002607
Train Epoch: 7 [3520/5000 (70%)]	Loss: 0.031238
Train Epoch: 7 [3680/5000 (74%)]	Loss: 0.009883
Train Epoch: 7 [3840/5000 (77%)]	Loss: 0.017177
Train Epoch: 7 [4000/5000 (80%)]	Loss: 0.265730
Train Epoch: 7 [4160/5000 (83%)]	Loss: 0.009637
Train Epoch: 7 [4320/5000 (86%)]	Loss: 0.008370
Train Epoch: 7 [4480/5000 (90%)]	Loss: 0.006234
Train Epoch: 7 [4640/5000 (93%)]	Loss: 0.034766
Train Epoch: 7 [4800/5000 (96%)]	Loss: 0.146087
Train Epoch: 7 [4960/5000 (99%)]	Loss: 0.008433
Test accuracy: 9516/10000 (95.000%)	Loss: 109.452637
Train Epoch: 8 [0/5000 (0%)]	Loss: 0.009398
Train Epoch: 8 [160/5000 (3%)]	Loss: 0.042391
Train Epoch: 8 [320/5000 (6%)]	Loss: 0.141988
Train Epoch: 8 [480/5000 (10%)]	Loss: 0.006414
Train Epoch: 8 [640/5000 (13%)]	Loss: 0.267083
Train Epoch: 8 [800/5000 (16%)]	Loss: 0.274849
Train Epoch: 8 [960/5000 (19%)]	Loss: 0.013489
Train Epoch: 8 [1120/5000 (22%)]	Loss: 0.001922
Train Epoch: 8 [1280/5000 (26%)]	Loss: 0.001376
Train Epoch: 8 [1440/5000 (29%)]	Loss: 0.004375
Train Epoch: 8 [1600/5000 (32%)]	Loss: 0.016826
Train Epoch: 8 [1760/5000 (35%)]	Loss: 0.007463
Train Epoch: 8 [1920/5000 (38%)]	Loss: 0.007734
Train Epoch: 8 [2080/5000 (42%)]	Loss: 0.000102
Train Epoch: 8 [2240/5000 (45%)]	Loss: 0.015938
Train Epoch: 8 [2400/5000 (48%)]	Loss: 0.100356
Train Epoch: 8 [2560/5000 (51%)]	Loss: 0.050858
Train Epoch: 8 [2720/5000 (54%)]	Loss: 0.001238
Train Epoch: 8 [2880/5000 (58%)]	Loss: 0.016850
Train Epoch: 8 [3040/5000 (61%)]	Loss: 0.077004
Train Epoch: 8 [3200/5000 (64%)]	Loss: 0.179450
Train Epoch: 8 [3360/5000 (67%)]	Loss: 0.039951
Train Epoch: 8 [3520/5000 (70%)]	Loss: 0.171324
Train Epoch: 8 [3680/5000 (74%)]	Loss: 0.031641
Train Epoch: 8 [3840/5000 (77%)]	Loss: 0.042311
Train Epoch: 8 [4000/5000 (80%)]	Loss: 0.007956
Train Epoch: 8 [4160/5000 (83%)]	Loss: 0.007650
Train Epoch: 8 [4320/5000 (86%)]	Loss: 0.004185
Train Epoch: 8 [4480/5000 (90%)]	Loss: 0.002937
Train Epoch: 8 [4640/5000 (93%)]	Loss: 0.002434
Train Epoch: 8 [4800/5000 (96%)]	Loss: 0.003249
Train Epoch: 8 [4960/5000 (99%)]	Loss: 0.002828
Test accuracy: 9615/10000 (96.000%)	Loss: 94.484734
Train Epoch: 9 [0/5000 (0%)]	Loss: 0.017549
Train Epoch: 9 [160/5000 (3%)]	Loss: 0.000493
Train Epoch: 9 [320/5000 (6%)]	Loss: 0.006262
Train Epoch: 9 [480/5000 (10%)]	Loss: 0.005245
Train Epoch: 9 [640/5000 (13%)]	Loss: 0.000083
Train Epoch: 9 [800/5000 (16%)]	Loss: 0.014870
Train Epoch: 9 [960/5000 (19%)]	Loss: 0.014329
Train Epoch: 9 [1120/5000 (22%)]	Loss: 0.001894
Train Epoch: 9 [1280/5000 (26%)]	Loss: 0.004105
Train Epoch: 9 [1440/5000 (29%)]	Loss: 0.029703
Train Epoch: 9 [1600/5000 (32%)]	Loss: 0.322544
Train Epoch: 9 [1760/5000 (35%)]	Loss: 0.001748
Train Epoch: 9 [1920/5000 (38%)]	Loss: 0.103174
Train Epoch: 9 [2080/5000 (42%)]	Loss: 0.009877
Train Epoch: 9 [2240/5000 (45%)]	Loss: 0.001582
Train Epoch: 9 [2400/5000 (48%)]	Loss: 0.001501
Train Epoch: 9 [2560/5000 (51%)]	Loss: 0.010855
Train Epoch: 9 [2720/5000 (54%)]	Loss: 0.014859
Train Epoch: 9 [2880/5000 (58%)]	Loss: 0.030137
Train Epoch: 9 [3040/5000 (61%)]	Loss: 0.064087
Train Epoch: 9 [3200/5000 (64%)]	Loss: 0.023796
Train Epoch: 9 [3360/5000 (67%)]	Loss: 0.018604
Train Epoch: 9 [3520/5000 (70%)]	Loss: 0.015594
Train Epoch: 9 [3680/5000 (74%)]	Loss: 0.091584
Train Epoch: 9 [3840/5000 (77%)]	Loss: 0.176761
Train Epoch: 9 [4000/5000 (80%)]	Loss: 0.672749
Train Epoch: 9 [4160/5000 (83%)]	Loss: 0.114453
Train Epoch: 9 [4320/5000 (86%)]	Loss: 0.001308
Train Epoch: 9 [4480/5000 (90%)]	Loss: 0.222034
Train Epoch: 9 [4640/5000 (93%)]	Loss: 0.015265
Train Epoch: 9 [4800/5000 (96%)]	Loss: 0.003146
Train Epoch: 9 [4960/5000 (99%)]	Loss: 0.009982
Test accuracy: 9588/10000 (95.000%)	Loss: 92.011047

In [ ]:
def main():
   device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Specifies where the torch.tensor is allocated
   trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
   name_model = "siamese_"
   extension_model = ".pt"
   
   model = Net().to(device)
   
   ##################
   #  training mode
   ##################
   if DO_LEARN: # training mode
      train_loader = torch.utils.data.DataLoader(Face_DS(train=True, transform=trans), batch_size=BATCH_SIZE, shuffle=True)
      test_loader = torch.utils.data.DataLoader(Face_DS(train=False, transform=trans), batch_size=BATCH_SIZE, shuffle=False)
      
      optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
      for epoch in range(NUM_EPOCH):
         train(model, device, train_loader, epoch, optimizer)
         test(model, device, test_loader)
         if epoch & save_frequency == 0:
            torch.save(model, (name_model + '{:03}' + extension_model).format(epoch))
            print("Model is saved!")
      
   ##################
   #  prediction
   ##################
   else: 
      dataset = Face_DS(train=False, transform=trans)
      #print(dataset.__repr__)
      prediction_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
      
      load_model_path = os.getcwd() + "/" + name_model + "000" + extension_model
      #model.load_state_dict(torch.load(load_model_path))
      model = torch.load(load_model_path)

      #####################################################################
      # Data: list containing the tensor representations of the 2 images
      #####################################################################
      data = []
      data.extend(next(iter(prediction_loader))[0][:3:2])
    
      # Using the getitem method 
      #all_indexes = list(range(0, dataset.__len__))
      #random.shuffle(all_indexes)
      #data.append(dataset.__getitem(all_indexes.pop(), visualization=True)) 
      #data.append(dataset.__getitem(all_indexes.pop(), visualization=True))  
       
      # Prediction
      same = oneshot(model, device, data)
      if same > 0:
         print('These two images are of the same number')
      else:
         print('These two images are not of the same number')


if __name__ == '__main__':
    #main()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Specifies where the torch.tensor is allocated
    trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])

    print("the type of trans is " + str(type(trans)))